# Group matches hcp

In [ ]:
import pandas as pd
import numpy as np

## Import file

In [ ]:
df = pd.read_csv('../../data/3. transformation/3_hcp_matches.csv', index_col = 0)

In [ ]:
df_data = df.copy()

df_data['group'] = 0
df_data['group'] = df_data['group'].astype(int)

group_counter = 1
for index, row in df_data.sort_values(['parent']).iterrows():
    #When group of parent = 0, create new group
    if df_data.loc[row['parent'], 'group'] == 0:
        if df_data.loc[index, 'group'] == 0:
            df_data.loc[index, 'group'] = group_counter
            group_counter += 1
        
        #Add Group to parent
        df_data.loc[df_data.loc[index, 'parent'], 'group'] = df_data.loc[index, 'group']
        #print("add group to parent: %s" % df_data.loc[row['parent'], 'group'])
    else:
        #take group of parent
        df_data.loc[index, 'group'] = df_data.loc[row['parent'], 'group']
    
    


## Set Main Address

In [4]:
# Import prioritylist
df_prioritylist = pd.read_csv('sources/prioritylist.csv')
df_prioritylist = df_prioritylist.set_index('source')

In [5]:
df_index = df_data.copy()

#Join groups and prioritylist
df_index = df_index.join(other=df_prioritylist, on='source')

#Prepare dataset
df_index = df_index.set_index(['group', df_index.index])
df_index['main_address'] = 0

#Loop Group
not_found_counter = 0
for index_group, df_group in df_index.groupby(level=0):
    
    if len(df_group) == 1:
        #Only one group member
        df_index.loc[index_group, 'main_address'] = 1
    else:
        #Find highest priority
        df_largest = df_group.nlargest(1, 'priority')
        df_index.loc[df_largest.iloc[0].name, 'main_address'] = 1
        if df_largest.iloc[0].priority == 0:
            #print("No priority found at: «%s», group: «%s»" % (df_largest.iloc[0]['name'], index_group))
            #print(df_group['source'].unique().tolist())
            not_found_counter += 1
            

print("No priority found: %s" % not_found_counter)



No priority found: 313


## Cleanup & Save

In [6]:
#Sort
#df_index = df_data.sort_values(['group', 'name'])

#Reset index
df_index = df_index.reset_index(drop=False)

#Clean
df_index.drop(axis=1, columns=['level_1', 'address_expand', 'location_expand', 'r_name', 'r_location', 'r_address', 'parent', 'r_ratio', 'log', 'priority'], inplace=True)

df_index.head()

#Save
df_index.to_csv('../../data/3. transformation/4_hcp_grouped.csv', index=False)

## Excel Export
Use this only for a preview, not for production

In [ ]:
import sys
sys.path.insert(0, '../../data/lib/')
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

#Sort
df_index = df_index.sort_values(['group', 'main_address'], ascending=[True, False])

df_index.head()


write_to_excel(df_index, 'tmp.xlsx', open=True, index=True)
